# ML workflow for  Chronic Kidney Disease

In [1]:
# Imports 
from pipeline_perso import *
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline

In [2]:
dtypes = {'pcv':'float64','wc':'float64','rc':'float64'}
data = pd.read_csv('kidney_disease.csv',dtype=dtypes,na_values='\t?')

In [3]:
data.reset_index(drop=True,inplace=True)
data.drop(['id'],axis=1,inplace=True)

In [4]:
TrainTest=TrainTestGenerator(1,test_size=0.3,data=data)
for train,test in TrainTest:
    X_train = np.array(data.drop(['classification'],axis=1))[train]
    X_test  = np.array(data.drop(['classification'],axis=1))[test]
    y_train = np.array(data.classification)[train]
    y_test  = np.array(data.classification)[test]

In [5]:
kf = KFold(n_splits=4, random_state=None, shuffle=True)

In [6]:
from catboost import CatBoostClassifier
parameters = {'depth': [6,8,10],'learning_rate' : [0.01, 0.05, 0.1],'iterations'    : [30, 50, 100]}

grid_perso = GridSearchHyperParamsCV(model=CatBoostClassifier(silent=True), parameters = parameters, cv_splitter = kf, n_jobs=-1 ,verbose=10)


pipe_perso = Pipeline([('imputer' , CustomImputer()),('cat_trans', CategoricalTransformer(strategy='ordinal_encoding')), ('catboost', grid_perso)])


pipe_perso.fit(X_train,y_train)


print(pipe_perso.score(X_test,y_test))
print(pipe_perso['catboost']._best_params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done  23 out of  27 | elapsed:   11.5s remaining:    2.0s
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:   16.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1627s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done  20 out of  27 | elapsed:    3.4s remaining:    1.2s
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    5.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.6s
[Parallel(n_j

0.975
{'depth': 8, 'learning_rate': 0.05, 'iterations': 50}
